<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/Phonetic_description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phonetic description practice application

In [ ]:
%%capture
!pip install gradio

In [ ]:
https://raw.githubusercontent.com/MK316/Myapps/refs/heads/main/data/Phonetic-description.csv

# Variable issue

# Revision (0923)

# Name included (0923): Final version

In [ ]:
#@markdown This app resolved variable issues (0923)

import random
import gradio as gr
import pandas as pd

# Load data from the provided URL
def load_data():
    url = "https://raw.githubusercontent.com/MK316/Myapps/refs/heads/main/data/Phonetic-description.csv"
    try:
        data = pd.read_csv(url, encoding='utf-8')  # Ensure correct encoding
        return data
    except Exception as e:
        raise ValueError(f"Error loading data: {e}")

# Load the data into a DataFrame
df = load_data()

# Dictionary to store each user's session-specific data
user_sessions = {}

# Function to generate a random question based on user selections
def generate_question(segment_type, description_type, used_questions):
    segment = "Consonant" if segment_type == "Consonant" else "Vowel"
    description_field = "Full_description" if description_type == "Full_description" else "Casual_description"

    filtered_df = df[df["Segment"] == segment]  # Filter based on consonant/vowel selection

    if filtered_df.empty:
        return "No data available", "", used_questions  # Handle empty filter case

    random_row = filtered_df.sample(1).iloc[0]  # Pick a random row
    description = random_row[description_field]  # Select either 'Full_description' or 'Casual_description'
    correct_ipa = random_row["IPA"]  # The correct IPA symbol
    used_questions.append(description)  # Track used descriptions to avoid repetition
    return description, correct_ipa, used_questions

# Function to check the user's answer
def submit_answer(user_ipa, correct_ipa, score, trials):
    trials += 1
    if user_ipa.strip() == correct_ipa:
        score += 1
        return f"Correct! The answer was '{correct_ipa}'", score, trials
    else:
        return f"Wrong! The correct answer was '{correct_ipa}'", score, trials

# Function to quit and show results
def quit_quiz(score, trials, name):
    return f"{name}! Your final score is {score}/{trials}."

# Gradio interface
def gradio_app():
    with gr.Blocks() as app:
        # Name input
        name_input = gr.Textbox(label="Enter your name", placeholder="Your name")

        # Radio buttons to select Consonant or Vowel
        segment_type = gr.Radio(choices=["Consonant", "Vowel"], label="Select Consonant or Vowel", value="Consonant")
        # Radio buttons to select description type (Full or Casual)
        description_type = gr.Radio(choices=["Full_description", "Casual_description"], label="Select Description Type", value="Full_description")

        # Start button renamed to "Quiz to start"
        start_button = gr.Button("Quiz to start")

        description_output = gr.Textbox(label="Description", interactive=False)
        ipa_input = gr.Textbox(label="Enter IPA", placeholder="Type IPA symbol here")
        submit_button = gr.Button("Submit", visible=False)  # Initially hidden until start is clicked
        result_output = gr.Textbox(label="Result", interactive=False)
        quit_button = gr.Button("Quit")

        # Start quiz and reset session data for the user
        def start(segment_type, description_type, name):
            # Initialize/reset user session
            user_sessions[name] = {
                "score": 0,
                "trials": 0,
                "used_questions": [],
                "correct_ipa": ""
            }
            description, correct_ipa, used_questions = generate_question(segment_type, description_type, user_sessions[name]["used_questions"])
            user_sessions[name]["correct_ipa"] = correct_ipa
            user_sessions[name]["used_questions"] = used_questions
            return description, "", gr.update(visible=True), user_sessions[name]["score"], user_sessions[name]["trials"]

        # Submit the answer and check
        def submit(name, user_ipa):
            session = user_sessions[name]
            result, score, trials = submit_answer(user_ipa, session["correct_ipa"], session["score"], session["trials"])
            session["score"] = score
            session["trials"] = trials

            # Generate new question after submitting the answer
            description, correct_ipa, used_questions = generate_question(segment_type.value, description_type.value, session["used_questions"])
            session["correct_ipa"] = correct_ipa
            session["used_questions"] = used_questions
            return description, "", result, gr.update(visible=True), result, score, trials

        # Quit and show final results
        def quit(name):
            session = user_sessions[name]
            return quit_quiz(session["score"], session["trials"], name)

        # Bind actions
        start_button.click(fn=start, inputs=[segment_type, description_type, name_input], outputs=[description_output, ipa_input, submit_button, result_output, result_output])
        submit_button.click(fn=submit, inputs=[name_input, ipa_input], outputs=[description_output, ipa_input, result_output, result_output, result_output])
        quit_button.click(fn=quit, inputs=[name_input], outputs=[result_output])

    return app

app = gradio_app()
app.launch()
